In [1]:
import numpy as np

In [2]:
input_file='./input_test.txt'
#input_file='./input.txt'

In [3]:
with open(input_file,'r') as f:
    lines=f.read().split('\n')

In [4]:
lines.remove('')

In [5]:
lines

['#.######', '#>>.<^<#', '#.<..<<#', '#>v.><>#', '#<^v^^>#', '######.#']

In [6]:
# First line tells initial position of elf, and width size of the area
elf_pos_initial=(-1,lines[0].index('.')-1) #y=-1 and x given
elf_pos_initial

(-1, 0)

In [7]:
elf_aim_position=(len(lines)-2,lines[-1].index('.')-1)
elf_aim_position

(4, 5)

In [8]:
valley_width = len(lines[0])-2
valley_height = len(lines)-2
print(valley_height)
print(valley_width)

4
6


Get initial state of blizzard

In [9]:
blizzard_up_initial=np.zeros((valley_height,valley_width), dtype=bool)
blizzard_down_initial=np.zeros((valley_height,valley_width), dtype=bool)
blizzard_right_initial=np.zeros((valley_height,valley_width), dtype=bool)
blizzard_left_initial=np.zeros((valley_height,valley_width), dtype=bool)
for irow,line in enumerate(lines[1:-1]):
    for icol, char in enumerate(line[1:-1]):
        if char=='^':
            blizzard_up_initial[irow,icol]=True
            continue
        if char=='>':
            blizzard_right_initial[irow,icol]=True
            continue
        if char=='v':
            blizzard_down_initial[irow,icol]=True
            continue
        if char=='<':
            blizzard_left_initial[irow,icol]=True

In order to find next movement, it is best to find the next blizzard position(s)

In [10]:
def getNextBlizzard(blizzard_up, blizzard_right, blizzard_down, blizzard_left):
    blizzard_up_next = np.zeros_like(blizzard_up)
    blizzard_up_next[0:-1,:] = blizzard_up[1:,:]
    blizzard_up_next[-1,:]=blizzard_up[0,:]

    blizzard_down_next = np.zeros_like(blizzard_down)
    blizzard_down_next[1:,:] = blizzard_down[0:-1,:]
    blizzard_down_next[0,:]=blizzard_down[-1,:]

    blizzard_right_next = np.zeros_like(blizzard_right)
    blizzard_right_next[:,1:] = blizzard_right[:,0:-1]
    blizzard_right_next[:,0]=blizzard_right[:,-1]

    blizzard_left_next = np.zeros_like(blizzard_left)
    blizzard_left_next[:,0:-1] = blizzard_left[:,1:]
    blizzard_left_next[:,-1]=blizzard_left[:,0]

    return (blizzard_up_next, blizzard_right_next, blizzard_down_next, blizzard_left_next)

In [11]:
def printBlizzardMap(blizzard_up, blizzard_right, blizzard_down, blizzard_left):
    irows,icols = blizzard_up.shape
    blizzards_total = blizzard_up.astype(np.uint8)+blizzard_right.astype(np.uint8)+blizzard_down.astype(np.uint8)+blizzard_left.astype(np.uint8)
    print('#'*(icols+2))
    for irow in range(irows):
        s='#'
        for icol in range(icols):
            bl_tot0=blizzards_total[irow,icol]
            if bl_tot0==0:
                s+='.'
                continue
            if bl_tot0>1:
                s+=str(blizzards_total[irow,icol])
                continue
            #Only one blizzard, put the right character
            if blizzard_up[irow,icol]:
                s+='^'
                continue
            if blizzard_right[irow,icol]:
                s+='>'
                continue
            if blizzard_down[irow,icol]:
                s+='v'
                continue
            if blizzard_left[irow,icol]:
                s+='<'
                continue
        s+='#'
        print(s)
    print('#'*(icols+2)) 

In [12]:
def get_manhatan_distance(vector0, vector1):
    v1=np.array(vector0)
    v2=np.array(vector1)

    return np.sum(np.abs(v1-v2))

Check Manh distance from start to end, for curiosity

In [13]:
no_blizz_distance= get_manhatan_distance(elf_pos_initial,elf_aim_position)
no_blizz_distance

10

Create a function that can work recursively that

Need global variable that has minimum minutes turns to get to destination from start.
global with dimension of valey

Returns (minimum) number of minutes to get to destination from this location. None if it could not find suitable route.

Parameters:
- next elf location
- next blizzard state
- current number of moves

In [14]:
global_minmins=None
global_max_mins=5*no_blizz_distance

In [15]:
global_elfpos_mins_history=[]

In [16]:
def get_minimum_minutes_from(elf_loc, blizz_state, n_cur_minutes):
    global valley_width
    global valley_height
    global elf_aim_position
    global global_minmins
    global global_max_mins
    global global_elfpos_mins_history
    
    print(f"elf_loc:{elf_loc}, n_cur_minutes:{n_cur_minutes}, global_minmins:{global_minmins}")
    if elf_loc[1]==elf_aim_position[1] and elf_loc[0]==elf_aim_position[0]-1:
        #Next move is to exit, regardless of blizzard
        print(f"Exit after:{n_cur_minutes+1}")
        return n_cur_minutes+1
    
    if (elf_loc,n_cur_minutes) in global_elfpos_mins_history:
        #already checked from this position
        return None
    else:
        global_elfpos_mins_history.append( (elf_loc,n_cur_minutes) )

    #Check current position against the global minmins
    if not global_minmins is None:
        if n_cur_minutes+get_manhatan_distance(elf_loc,elf_aim_position) > global_minmins:
            #No point trying to find a route as it will take more minutes than minimum set
            return None
    
    if n_cur_minutes>global_max_mins:
        return None
        
    #Update blizzard to new state
    blizz_next = getNextBlizzard(*blizz_state)

    #with this new blizzard check possible movements
    next_positions=[]
    #Preference down or right

    bl0,bl1,bl2,bl3= blizz_next
    #Blizard any
    bl_any = bl0 | bl1 | bl2 | bl3

    #Check right
    if elf_loc[1]<valley_width-1:
        pos1= (elf_loc[0],elf_loc[1]+1)
        if not bl_any[pos1[0], pos1[1]]:
            next_positions.append(pos1)

    #Check down
    if elf_loc[0]<valley_height-1:
        pos1=(elf_loc[0]+1,elf_loc[1])
        if not bl_any[pos1[0], pos1[1]]:
            next_positions.append(pos1)
    

    #Check left
    if elf_loc[1]>0:
        pos1= (elf_loc[0],elf_loc[1]-1)
        if not bl_any[pos1[0], pos1[1]]:
            next_positions.append(pos1)
    
    #Check stationary
    pos1= (elf_loc[0],elf_loc[1])
    if not bl_any[pos1[0], pos1[1]]:
        next_positions.append(pos1)

    #Check up
    if elf_loc[0]>0:
        pos1=(elf_loc[0]-1,elf_loc[1])
        if not bl_any[pos1[0], pos1[1]]:
            next_positions.append(pos1)
    
    #At this point next_positions should have all possible positions to try if any
    if len(next_positions)==0:
        #No movements of stationary is possible, returning None
        return None
    else:
        n_cur_minutes0=n_cur_minutes+1 #for next iteration
        for nextpos0 in next_positions:
            mins=get_minimum_minutes_from(nextpos0, blizz_next, n_cur_minutes0)

            #Process mins
            if not mins is None:
                if global_minmins is None:
                    global_minmins=mins
                else:
                    global_minmins=min(global_minmins,mins)
        
        return global_minmins

# try running

In [17]:
blizz_start = (blizzard_up_initial, blizzard_right_initial, blizzard_down_initial, blizzard_left_initial)

In [18]:
global_minmins=None
global_max_mins= 8*no_blizz_distance
print(f"global_max_mins:{global_max_mins}")
global_elfpos_mins_history=[]
get_minimum_minutes_from(elf_pos_initial,blizz_start,0)

global_max_mins:80
elf_loc:(-1, 0), n_cur_minutes:0, global_minmins:None
elf_loc:(0, 0), n_cur_minutes:1, global_minmins:None
elf_loc:(1, 0), n_cur_minutes:2, global_minmins:None
elf_loc:(1, 0), n_cur_minutes:3, global_minmins:None
elf_loc:(0, 0), n_cur_minutes:4, global_minmins:None
elf_loc:(0, 1), n_cur_minutes:5, global_minmins:None
elf_loc:(0, 2), n_cur_minutes:6, global_minmins:None
elf_loc:(1, 2), n_cur_minutes:7, global_minmins:None
elf_loc:(1, 1), n_cur_minutes:8, global_minmins:None
elf_loc:(1, 0), n_cur_minutes:9, global_minmins:None
elf_loc:(0, 0), n_cur_minutes:10, global_minmins:None
elf_loc:(0, 1), n_cur_minutes:9, global_minmins:None
elf_loc:(0, 2), n_cur_minutes:10, global_minmins:None
elf_loc:(0, 2), n_cur_minutes:11, global_minmins:None
elf_loc:(1, 2), n_cur_minutes:12, global_minmins:None
elf_loc:(2, 2), n_cur_minutes:13, global_minmins:None
elf_loc:(2, 3), n_cur_minutes:14, global_minmins:None
elf_loc:(2, 4), n_cur_minutes:15, global_minmins:None
elf_loc:(2, 5), n_c

18

`input_test.txt` gives correct answer of 18

`input.txt` not 626, 586, 561, 489, 362. Correct answer 286


# Check going back after reaching end

Need to get blizzard state after 286 minutes

set initial position to be the current end, and the end poition the opposite

In [19]:
import time
def get_minimum_minutes_from_back(elf_loc, blizz_state, n_cur_minutes, elf_aim_pos, progress0,progressDiv):
    global valley_width
    global valley_height
    global global_minmins
    global global_max_mins
    global global_elfpos_mins_history
    global t0
    
    t1=time.time()
    if t1-t0>60: #Check progress every minute
        print(f"elf_loc:{elf_loc}, n_cur_minutes:{n_cur_minutes}, global_minmins:{global_minmins}, progress:{progress0}")
        t0=t1
    
    mdist = get_manhatan_distance(elf_loc,elf_aim_pos)

    if mdist==1:
        #Next move is to exit, regardless of blizzard
        print(f"Exit after:{n_cur_minutes+1}")
        print(f"elf_loc:{elf_loc}, n_cur_minutes:{n_cur_minutes}, global_minmins:{global_minmins}, progress:{progress0}")
        return n_cur_minutes+1

    if (elf_loc,n_cur_minutes) in global_elfpos_mins_history:
        #already checked from this position
        return None
    else:
        global_elfpos_mins_history.append( (elf_loc,n_cur_minutes) )

    #Check current position against the global minmins
    if not global_minmins is None:
        if n_cur_minutes+get_manhatan_distance(elf_loc,elf_aim_pos) > global_minmins:
            #No point trying to find a route as it will take more minutes than minimum set
            return None
    
    if n_cur_minutes>global_max_mins:
        return None
        
    #Update blizzard to new state
    blizz_next = getNextBlizzard(*blizz_state)

    #with this new blizzard check possible movements
    next_positions=[]
    #Preference down or right

    bl0,bl1,bl2,bl3= blizz_next
    #Blizard any
    bl_any = bl0 | bl1 | bl2 | bl3

    #Check left
    if elf_loc[1]>0 and elf_loc[0]<valley_height:
        pos1= (elf_loc[0],elf_loc[1]-1)
        if not bl_any[pos1[0], pos1[1]]:
            next_positions.append(pos1)
    
    #Check up
    if elf_loc[0]>0:
        pos1=(elf_loc[0]-1,elf_loc[1])
        if not bl_any[pos1[0], pos1[1]]:
            next_positions.append(pos1)

    #Check stationary
    if elf_loc[0]<valley_height and elf_loc[1]<valley_width:
        pos1= (elf_loc[0],elf_loc[1])
        if not bl_any[pos1[0], pos1[1]]:
            next_positions.append(pos1)
    
    #Check right
    if elf_loc[1]<valley_width-1:
        pos1= (elf_loc[0],elf_loc[1]+1)
        if not bl_any[pos1[0], pos1[1]]:
            next_positions.append(pos1)

    #Check down
    if elf_loc[0]<valley_height-1:
        pos1=(elf_loc[0]+1,elf_loc[1])
        if not bl_any[pos1[0], pos1[1]]:
            next_positions.append(pos1)

    
    #At this point next_positions should have all possible positions to try if any
    if len(next_positions)==0:
        #No movements of stationary is possible, returning None
        return None
    else:
        n_cur_minutes0=n_cur_minutes+1 #for next iteration
        progressDiv1=progressDiv/len(next_positions)
        for nextpos0 in next_positions:
            mins=get_minimum_minutes_from_back(nextpos0, blizz_next, n_cur_minutes0, elf_aim_pos, progress0, progressDiv1)

            #Process mins
            if not mins is None:
                if global_minmins is None:
                    global_minmins=mins
                else:
                    global_minmins=min(global_minmins,mins)
            progress0+=progressDiv1
        return global_minmins

In [20]:
bliz18=blizz_start

for i in range(18):
    bliz286=getNextBlizzard(*bliz18)

In [21]:
elf_aim_position_back=elf_pos_initial
elf_pos_initial_back=elf_aim_position

In [22]:
global_minmins=None
global_max_mins= 5*no_blizz_distance
print(f"global_max_mins:{global_max_mins}")
global_elfpos_mins_history=[]

progress=0
t0=time.time()
get_minimum_minutes_from_back(elf_pos_initial_back,bliz286,0, elf_aim_position_back,0.0,100.0)

global_max_mins:50
Exit after:16
elf_loc:(0, 0), n_cur_minutes:15, global_minmins:None, progress:4.166666666666667


16

Answer for coming back is 16

In [23]:
18+16

34

So total blizzard state is 255+286=541

# Going down again

In [78]:
import time
def get_minimum_minutes_from_downagain(elf_loc, blizz_state, n_cur_minutes, elf_aim_pos, progress0,progressDiv):
    global valley_width
    global valley_height
    global global_minmins
    global global_max_mins
    global global_elfpos_mins_history
    global t0
    
    t1=time.time()
    if t1-t0>60: #Check progress every minute
        print(f"elf_loc:{elf_loc}, n_cur_minutes:{n_cur_minutes}, global_minmins:{global_minmins}, progress:{progress0}")
        t0=t1
    
    mdist = get_manhatan_distance(elf_loc,elf_aim_pos)

    if mdist==1:
        #Next move is to exit, regardless of blizzard
        print(f"Exit after:{n_cur_minutes+1}")
        print(f"elf_loc:{elf_loc}, n_cur_minutes:{n_cur_minutes}, global_minmins:{global_minmins}, progress:{progress0}")
        return n_cur_minutes+1

    if (elf_loc,n_cur_minutes) in global_elfpos_mins_history:
        #already checked from this position
        return None
    else:
        global_elfpos_mins_history.append( (elf_loc,n_cur_minutes) )

    #Check current position against the global minmins
    if not global_minmins is None:
        if n_cur_minutes+get_manhatan_distance(elf_loc,elf_aim_pos) > global_minmins:
            #No point trying to find a route as it will take more minutes than minimum set
            return None
    
    if n_cur_minutes>global_max_mins:
        return None
        
    #Update blizzard to new state
    blizz_next = getNextBlizzard(*blizz_state)

    #with this new blizzard check possible movements
    next_positions=[]
    #Preference down or right

    bl0,bl1,bl2,bl3= blizz_next
    #Blizard any
    bl_any = bl0 | bl1 | bl2 | bl3

    #Check right
    if elf_loc[1]<valley_width-1:
        pos1= (elf_loc[0],elf_loc[1]+1)
        if not bl_any[pos1[0], pos1[1]]:
            next_positions.append(pos1)
            
    #Check down
    if elf_loc[0]<valley_height-1:
        pos1=(elf_loc[0]+1,elf_loc[1])
        if not bl_any[pos1[0], pos1[1]]:
            next_positions.append(pos1)

    #Check stationary
    if elf_loc[0]<valley_height and elf_loc[1]<valley_width:
        pos1= (elf_loc[0],elf_loc[1])
        if not bl_any[pos1[0], pos1[1]]:
            next_positions.append(pos1)

    #Check left
    if elf_loc[1]>0 and elf_loc[0]<valley_height:
        pos1= (elf_loc[0],elf_loc[1]-1)
        if not bl_any[pos1[0], pos1[1]]:
            next_positions.append(pos1)
    
    #Check up
    if elf_loc[0]>0:
        pos1=(elf_loc[0]-1,elf_loc[1])
        if not bl_any[pos1[0], pos1[1]]:
            next_positions.append(pos1)
    
    #At this point next_positions should have all possible positions to try if any
    if len(next_positions)==0:
        #No movements of stationary is possible, returning None
        return None
    else:
        n_cur_minutes0=n_cur_minutes+1 #for next iteration
        progressDiv1=progressDiv/len(next_positions)
        for nextpos0 in next_positions:
            mins=get_minimum_minutes_from_downagain(nextpos0, blizz_next, n_cur_minutes0, elf_aim_pos, progress0, progressDiv1)

            #Process mins
            if not mins is None:
                if global_minmins is None:
                    global_minmins=mins
                else:
                    global_minmins=min(global_minmins,mins)
            progress0+=progressDiv1
        return global_minmins

In [79]:
bliz541=blizz_start

for i in range(541):
    bliz541=getNextBlizzard(*bliz541)

In [80]:
print(elf_pos_initial)
print(elf_aim_position)

(-1, 0)
(25, 119)


In [81]:
global_minmins=None
global_max_mins= 5*no_blizz_distance
print(f"global_max_mins:{global_max_mins}")
global_elfpos_mins_history=[]

progress=0
t0=time.time()
get_minimum_minutes_from_downagain(elf_pos_initial,bliz541,0, elf_aim_position,0.0,100.0)

global_max_mins:725
elf_loc:(2, 80), n_cur_minutes:682, global_minmins:None, progress:44.5601951080882
Exit after:715
elf_loc:(24, 119), n_cur_minutes:714, global_minmins:None, progress:44.5601951080882
Exit after:696
elf_loc:(24, 119), n_cur_minutes:695, global_minmins:715, progress:44.5601951080882
Exit after:696
elf_loc:(24, 119), n_cur_minutes:695, global_minmins:696, progress:44.5601951080882
elf_loc:(16, 89), n_cur_minutes:618, global_minmins:696, progress:44.5601951080882
Exit after:685
elf_loc:(24, 119), n_cur_minutes:684, global_minmins:696, progress:44.5601951080882
Exit after:656
elf_loc:(24, 119), n_cur_minutes:655, global_minmins:685, progress:44.5601951080882
Exit after:645
elf_loc:(24, 119), n_cur_minutes:644, global_minmins:656, progress:44.5601951080882
Exit after:645
elf_loc:(24, 119), n_cur_minutes:644, global_minmins:645, progress:44.5601951080882
Exit after:639
elf_loc:(24, 119), n_cur_minutes:638, global_minmins:645, progress:44.5601951080882
elf_loc:(15, 66), n_c